<a href="https://colab.research.google.com/github/MiguelG26/Public_Projects/blob/master/db-covid19-worldwide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd #
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
import datetime
from datetime import date

In [ ]:
#new data on a weekly basis
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv/data.csv')

df.columns=['countriesAndTerritories', 'countryterritoryCode', 'continent', 'popData2019', 'indicator',
       'weekly_count', 'year_week', 'rate_14_day', 'cumulative_count','source','note']

df=df[['year_week','countriesAndTerritories', 'countryterritoryCode', 'popData2019', 'continent' , 'indicator','weekly_count']]

cases=list(df[df['indicator']=="cases"]['weekly_count'])
df=df[df['indicator']=="deaths"]
df['cases']=cases 
df = df.rename(columns={'weekly_count': 'deaths','continent': 'continentExp'})

df['dateRep']=df.apply(lambda x: datetime.datetime.strptime(x['year_week'] + '-1', "%Y-%W-%w"),axis=1)
df['year']=df['dateRep'].dt.year

mymap={1:'01',2:'02',3:'03',4:'04',5:'05',6:'06',7:'07',8:'08',9:'09',10:'10',11:'11',12:'12'}
df['month']=df['dateRep'].dt.month.map(mymap)
df['year_month']=df['year'].astype(str)+"_"+df['month']
df['month']=df['dateRep'].dt.month #to bring the column "month" back to integer
df['week']=df['year_week'].str[5:].astype(int)
df=df[['dateRep','year', 'month', 'week', 'year_month' , 'cases','deaths','countriesAndTerritories','popData2019','continentExp']]

#to remove the rows including aggregated data from continents
no_countries=['Africa (total)','America (total)','Asia (total)','EU/EEA (total)','Europe (total)','Oceania (total)']
df= df[~df['countriesAndTerritories'].isin (no_countries)]

data=df.copy()
data

,dateRep,year,month,week,year_month,cases,deaths,countriesAndTerritories,popData2019,continentExp
105,2020-01-06,2020,1,1,2020_01,0.0,0.0,Afghanistan,38928341,Asia
106,2020-01-13,2020,1,2,2020_01,0.0,0.0,Afghanistan,38928341,Asia
107,2020-01-20,2020,1,3,2020_01,0.0,0.0,Afghanistan,38928341,Asia
108,2020-01-27,2020,1,4,2020_01,0.0,0.0,Afghanistan,38928341,Asia
109,2020-02-03,2020,2,5,2020_02,0.0,0.0,Afghanistan,38928341,Asia
...,...,...,...,...,...,...,...,...,...,...
43101,2021-11-29,2021,11,48,2021_11,5055.0,5.0,Zimbabwe,14862927,Africa
43102,2021-12-06,2021,12,49,2021_12,28094.0,28.0,Zimbabwe,14862927,Africa
43103,2021-12-13,2021,12,50,2021_12,25656.0,49.0,Zimbabwe,14862927,Africa
43104,2021-12-27,2021,12,52,2021_12,NaN,NaN,Zimbabwe,14862927,Africa


In [ ]:
print("Total deaths: " + '{:,.0f}'.format(data['deaths'].sum()))
print("Total cases: " + '{:,.0f}'.format(data['cases'].sum()))
print("Mortality rate: " + str(round(data['deaths'].sum()/data['cases'].sum()*100,1))+"%")
print(data['dateRep'].max())

Total deaths: 5,427,885
Total cases: 297,372,385
Mortality rate: 1.8%
2022-01-03 00:00:00


In [ ]:
df0=data.groupby('year_month')[['cases','deaths']].sum()
fig = px.bar(x=df0.index, y=df0['deaths'], labels={'x':'Date', 'y':'Cases'},height=400, width=1200, title='New cases per day WW')
fig.show()

In [ ]:
#######################################################################
#Data Preparation
#######################################################################
df=data.groupby(['countriesAndTerritories','continentExp'])[['cases','deaths']].sum().sort_values('cases',ascending=False)
df =df.reset_index()

df.index=df['countriesAndTerritories']
df=df.drop('countriesAndTerritories', axis=1)

dfpop=data.groupby('countriesAndTerritories')[['popData2019']].mean().round(0) # to get the population per country
df=pd.concat([df,dfpop],axis=1)

dfpop=df.groupby('continentExp')[['popData2019']].sum().round(0)
df=df.groupby(['continentExp'])[['cases','deaths']].sum()
df=pd.concat([df,dfpop],axis=1)

df['%All_Cases']=(df['cases']/df['cases'].sum()*100).round(2)
df['%All_Deaths']=(df['deaths']/df['deaths'].sum()*100).round(2)
df['Mortality_rate']=(df['deaths']/df['cases']*100).round(2)

df=df[['popData2019','cases', 'deaths', '%All_Cases', '%All_Deaths','Mortality_rate']]

df['Deaths_by_population']=(df['deaths']/df['popData2019']*100).round(5)
df['Cases_by_population']=(df['cases']/df['popData2019']*100).round(5)

df=df.sort_values('cases',ascending=False)
#df=df.drop('Other',axis=0)
#######################################################################
#Data Visualization
#######################################################################

fig = make_subplots(rows=2, cols=3,subplot_titles=("<b>Deaths</b> per Continent due to Covid-19"+ '<br> '+ '<br> ',
                                                   " ",
                                                   "<b>Cases</b> per Continent due to Covid-19"+ '<br> '+ '<br> ',
                                                   "% of <b>Deaths</b> by population"+ '<br> '+ '<br> ',
                                                   " ",
                                                   "% of <b>Cases</b> by population"+ '<br> '+ '<br> ',))
#Graph1
df1=df.sort_values('deaths',ascending=False).head(10)
df1=df1.sort_values('deaths',ascending=True)
fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['deaths'],text=df1['deaths'],orientation="h",marker=dict(color='#b1b1b5')),
    row=1, col=1,
)

#Graph2
df1=df.sort_values('cases',ascending=False).head(10)
df1=df1.sort_values('cases',ascending=True)
fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['cases'],text=df1['cases'],orientation="h",marker_color='#385d7f'),
    row=1, col=3
)

fig.update_traces(texttemplate='%{text:.2s}')

#Graph3
df1=df.sort_values('Deaths_by_population',ascending=False).head(5)
df1=df1.sort_values('Deaths_by_population',ascending=True)
fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['Deaths_by_population'],text=df1['Deaths_by_population'].round(3),orientation="h",marker_color='#b1b1b5'),
    row=2, col=1
)

#Graph4
df1=df.sort_values('Cases_by_population',ascending=False).head(10)
df1=df1.sort_values('Cases_by_population',ascending=True)
fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['Cases_by_population'],text=df1['Cases_by_population'].round(2),orientation="h",marker_color='#385d7f'),
    row=2, col=3
)

fig.update_layout(height=900, width=1200,template="simple_white", showlegend=False,
                  xaxis =dict(showticklabels=False,showline=False,ticks='' ),
                  yaxis =dict(showline=False, zeroline = False,ticks=''),
                  xaxis3 =dict(showticklabels=False,showline=False,ticks=''),
                  yaxis3 =dict(showline=False, zeroline = False,ticks=''),
                  xaxis4 =dict(side='top'),
                  yaxis4 =dict(showline=False, zeroline = False,ticks=''),
                  xaxis6 =dict(side='top'),
                  yaxis6 =dict(showline=False, zeroline = False,ticks=''))

fig.update_traces(textposition='auto')

fig.show()
df

,popData2019,cases,deaths,%All_Cases,%All_Deaths,Mortality_rate,Deaths_by_population,Cases_by_population
continentExp,,,,,,,,
America,1021703563,106617085.0,2405673.0,35.85,44.32,2.26,0.23546,10.43523
Europe,851192161,106224901.0,1647158.0,35.72,30.35,1.55,0.19351,12.47954
Asia,4498460442,74113423.0,1141789.0,24.92,21.04,1.54,0.02538,1.64753
Africa,1339423921,9554514.0,228463.0,3.21,4.21,2.39,0.01706,0.71333
Oceania,42515205,862462.0,4802.0,0.29,0.09,0.56,0.01129,2.02860


In [ ]:
#######################################################################
#Data Preparation
#######################################################################
country='Austria' #'United_States_of_America'
head_quantity=10
#######################################################################
df=data.groupby(['countriesAndTerritories','continentExp'])[['cases','deaths']].sum().sort_values('cases',ascending=False)
df =df.reset_index()
df.index=df['countriesAndTerritories']
df=df.drop('countriesAndTerritories', axis=1)

dfpop=data.groupby('countriesAndTerritories')[['popData2019']].mean().round(0)
df['%All_Cases']=(df['cases']/df['cases'].sum()*100).round(2)
df['%All_Deaths']=(df['deaths']/df['deaths'].sum()*100).round(2)
df['Mortality_rate']=(df['deaths']/df['cases']*100).round(2)

df=pd.concat([df,dfpop],axis=1)
df['Deaths_by_population']=(df['deaths']/df['popData2019']*100).round(5)
df['Cases_by_population']=(df['cases']/df['popData2019']*100).round(5)

df=df.dropna()
df=df[df['popData2019']>999999]

#df=df[df['continentExp']=="Europe"] # Modify if needed!

#df=df[df.index.isin(['Austria','Netherlands','Germany','Belgium', 'Italy'])] # Modify if needed!
#######################################################################
#Data Visualization
#######################################################################

fig = make_subplots(rows=2, cols=3,subplot_titles=("<b>Deaths</b> per Country due to Covid-19"+ '<br> '+ '<br> ',
                                                   " ",
                                                   "<b>Cases</b> per Country of Covid-19"+ '<br> '+ '<br> ',
                                                   "% of <b>Deaths</b> by population"+ '<br> '+ '<br> ',
                                                   " ",
                                                   "% of <b>Cases</b> by population"+ '<br> '+ '<br> ',))

#Graph1 Deaths
df1=df.sort_values('deaths',ascending=False).head(head_quantity)
countries1=list(df1.head(5).index) #This is needed in the chart below!
if sum(df1.index==country)==0:
  df1=df1.append(df[df.index==country])
df1=df1.sort_values('deaths',ascending=True)

fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['deaths'],text=df1['deaths'],orientation="h",marker_color='#b1b1b5'),
    row=1, col=1
)
x=df1
#Graph2 Cases
df1=df.sort_values('cases',ascending=False).head(head_quantity)
countries2=list(df1.head(5).index) #This is needed in the chart below!
if sum(df1.index==country)==0:
  df1=df1.append(df[df.index==country])
df1=df1.sort_values('cases',ascending=True)

fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['cases'],text=df1['cases'],orientation="h",marker_color='#385d7f'),
    row=1, col=3
)

fig.update_traces(texttemplate='%{text:.2s}')

#Graph3 Deaths_by_population
df1=df.sort_values('Deaths_by_population',ascending=False).head(head_quantity)
countries3=list(df1.head(5).index) #This is needed in the chart below!
if sum(df1.index==country)==0:
  df1=df1.append(df[df.index==country])
df1=df1.sort_values('Deaths_by_population',ascending=True)

fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['Deaths_by_population'],text=df1['Deaths_by_population'].round(2),orientation="h",marker_color='#b1b1b5'),
    row=2, col=1
)

#Graph4 Cases_by_population
df1=df.sort_values('Cases_by_population',ascending=False).head(head_quantity)
countries4=list(df1.head(5).index) #This is needed in the chart below!
if sum(df1.index==country)==0:
  df1=df1.append(df[df.index==country])
df1=df1.sort_values('Cases_by_population',ascending=True)

fig.add_trace(
    go.Bar(y=df1.index +"  ", x=df1['Cases_by_population'],text=df1['Cases_by_population'].round(2),orientation="h",marker_color='#385d7f'),
    row=2, col=3
)

#Layout
fig.update_layout(height=900, width=1200,template="simple_white", showlegend=False,
                  xaxis =dict(showticklabels=False,showline=False,ticks=''),
                  yaxis =dict(showline=False, zeroline = False,ticks=''),
                  xaxis3 =dict(showticklabels=False,showline=False,ticks=''),
                  yaxis3 =dict(showline=False, zeroline = False,ticks=''),
                  xaxis4 =dict(side='top'),
                  yaxis4 =dict(showline=False, zeroline = False,ticks=''),
                  xaxis6 =dict(side='top'),
                  yaxis6 =dict(showline=False, zeroline = False,ticks=''))

fig.update_traces(textposition='auto')


fig.show()
#x

In [ ]:
#######################################################################
#Setting the time interval
#######################################################################
month =12
year=2021
#######################################################################
#Data Preparation
#######################################################################
mymap={1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'Mai',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

x=data[(data['year']==year) & (data['month']==month)].copy()
x=x.groupby(['countriesAndTerritories'])[['cases','deaths']].sum()

y=df['popData2019'].copy()
x=pd.concat([x,y],axis=1)
x=x[x['popData2019']>999999]############

x['new_deaths_rate']=x['deaths']/x['popData2019']*100
x['new_cases_rate']=x['cases']/x['popData2019']*100

#######################################################################
#Data Visualization
#######################################################################

fig = make_subplots(rows=1, cols=3,subplot_titles=("<b>New Deaths</b> in "+"<b>{}</b>".format(mymap[month]) +
                                                   '<br>' + 'in % of pop per Country'
                                                   '<br> ',
                                                   " ",
                                                   "<b>New Cases</b> in "+"<b>{}</b>".format(mymap[month]) +
                                                   '<br>' + 'in % of pop per Country'
                                                   '<br> ',
                                                   ))


#Graph 1 Deaths
x1=x.sort_values('new_deaths_rate',ascending=False).head(20)
x1=x1.sort_values('new_deaths_rate')
fig.add_trace(
    go.Bar(y=x1.index +"  ", x=x1['new_deaths_rate'],text=x1['new_deaths_rate'].round(2),orientation="h",marker_color='#385d7f'),
    row=1, col=1
)

#Graph2 Cases
x2=x.sort_values('new_cases_rate',ascending=False).head(20)
x2=x2.sort_values('new_cases_rate')

fig.add_trace(
    go.Bar(y=x2.index +"  ", x=x2['new_cases_rate'],text=x2['new_cases_rate'].round(2),orientation="h",marker_color='#b1b1b5'),
    row=1, col=3
)


#Layout
fig.update_layout(height=900, width=1200,template="simple_white", showlegend=False,
                  xaxis =dict(showticklabels=False,showline=False,ticks=''),
                  yaxis =dict(showline=False, zeroline = False,ticks=''),
                  xaxis3 =dict(showticklabels=False,showline=False,ticks=''),
                  yaxis3 =dict(showline=False, zeroline = False,ticks=''),)

fig.update_traces(textposition='auto')

fig.show()


In [ ]:
colors=list(px.colors.qualitative.G10)
colors=list(px.colors.sequential.Blues)[1:]
#colors=list(px.colors.diverging.delta)
#colors=list(px.colors.cyclical.IceFire)
#https://plotly.com/python/builtin-colorscales/

country = 'Germany'
countries1=list(x1.tail(3).index)[::-1] #[start:stop:step]
countries2=list(x2.tail(3).index)[::-1] #[start:stop:step]

#countries1=['Italy','Spain','Germany','Austria'] #Modify inf needed
#countries2=['Italy','Spain','Germany','Austria'] #Modify inf needed

dfc=data.copy()
dfc=dfc[dfc['dateRep']>'2020-05-31']#!!!!!!!!!!!!!!!!

dfc=dfc.groupby(['year_month','countriesAndTerritories'])[['cases','deaths']].sum()
dfc =dfc.reset_index()

#mymap={1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'Mai',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
dfc.index=dfc['year_month']
dfc=dfc.drop('year_month', axis=1)

############
#Deaths
############
fig = go.Figure()
for h,i in enumerate(countries1):
  dfcc=dfc[dfc['countriesAndTerritories']==i]
  pop=df['popData2019'][df.index==i][0]
  y=(dfcc['deaths']/pop*100).round(7)
  if i==country:
    fig.add_trace(go.Scatter(x=dfcc.index, y=y,mode='lines',name=i,line=dict(color='#c94f5f',width=6,dash='dot')))
    #y=y
  else:
    fig.add_trace(go.Scatter(x=dfcc.index, y=y,mode='lines',name=i,line=dict(color=colors[7-h*2],width=7-h)))

if countries1.count(country)==0:
  dfcc=dfc[dfc['countriesAndTerritories']==country]
  pop=df['popData2019'][df.index==country][0]
  y=(dfcc['deaths']/pop*100).round(7)
  fig.add_trace(go.Scatter(x=dfcc.index, y=y,mode='lines',name=country,line=dict(color='#c94f5f',width=3,dash='dot')))

fig.update_layout(height=400, width=1400,template="simple_white",title_x=0.4,
                  #title_text='<span style="font-size: 18px;color:#737375"> <b>April</b> was horrible <b>Worldwide!</b> and <b>July</b> too but mainly in <b>America</b></span>'+ '<br>'+'',
                  yaxis =dict(showline=False, zeroline = False, title='% of <b>Deaths</b> by population'),
                  )
fig.show()

############
#Cases
############
fig = go.Figure()
for h,i in enumerate(countries2):
  dfcc=dfc[dfc['countriesAndTerritories']==i]
  pop=df['popData2019'][df.index==i][0]
  y=(dfcc['cases']/pop*100).round(7)
  if i==country:
    fig.add_trace(go.Scatter(x=dfcc.index, y=y,mode='lines',name=i,line=dict(color='#c94f5f',width=3,dash='dot')))
  else:
    fig.add_trace(go.Scatter(x=dfcc.index, y=y,mode='lines',name=i,line=dict(color=colors[7-h*2],width=7-h)))

if countries2.count(country)==0:
  dfcc=dfc[dfc['countriesAndTerritories']==country]
  pop=df['popData2019'][df.index==country][0]
  y=(dfcc['cases']/pop*100).round(7)
  fig.add_trace(go.Scatter(x=dfcc.index, y=y,mode='lines',name=country,line=dict(color='#c94f5f',width=3,dash='dot')))

fig.update_layout(height=400, width=1400,template="simple_white",title_x=0.4,
                  #title_text='<span style="font-size: 18px;color:#737375">Finally the world started testing more in July!</span>'+ '<br>'+'',
                  yaxis =dict(showline=False, zeroline = False, title='% of <b>Cases</b> by population'),
                  )
fig.show()


In [ ]:
country='Croatia'

df=data.groupby(['countriesAndTerritories','continentExp'])[['cases','deaths']].sum().sort_values('cases',ascending=False)
df =df.reset_index()
df.index=df['countriesAndTerritories']
df=df.drop('countriesAndTerritories', axis=1)

dfpop=data.groupby('countriesAndTerritories')[['popData2019']].mean().round(0)
df['%All_Cases']=(df['cases']/df['cases'].sum()*100).round(2)
df['%All_Deaths']=(df['deaths']/df['deaths'].sum()*100).round(2)
df['Mortality_rate']=(df['deaths']/df['cases']*100).round(2)

df=pd.concat([df,dfpop],axis=1)
df['Deaths_by_population']=(df['deaths']/df['popData2019']*100).round(5)
df['Cases_by_population']=(df['cases']/df['popData2019']*100).round(5)

df=df.dropna()
####################################################
print('Country: ' + country)
print("")
print(df.loc[country])
print("")
print("Days with highest quantity of deaths in " + str(country)+":")
print("")
print(data[['dateRep','cases','deaths']][data['countriesAndTerritories']==country].sort_values('cases',ascending=False).head(5))

data1=data.sort_index(ascending=False).copy()
x=data1['dateRep'][data1['countriesAndTerritories']==country]

y=data1['deaths'][data1['countriesAndTerritories']==country]
fig = px.bar(x=x, y=y, labels={'x':'Date', 'y':'Deaths'}, title='Deaths per day in '+country,height=300, width=1200)
fig.show()

y=data1['cases'][data1['countriesAndTerritories']==country]
fig = px.bar(x=x, y=y, labels={'x':'Date', 'y':'Cases'}, title='New Cases per day in '+country,height=300, width=1200)
fig.show()

Country: Croatia

continentExp             Europe
cases                    767183
deaths                    12918
%All_Cases                 0.26
%All_Deaths                0.24
Mortality_rate             1.68
popData2019             4058165
Deaths_by_population    0.31832
Cases_by_population     18.9047
Name: Croatia, dtype: object

Days with highest quantity of deaths in Croatia:

         dateRep    cases  deaths
10033 2022-01-03  45129.0   265.0
10025 2021-11-08  37318.0   390.0
10027 2021-11-22  32640.0   450.0
10026 2021-11-15  31820.0   440.0
10024 2021-11-01  31095.0   295.0
